# 🩻 Detección de Neumonía con IA en Rayos X
Este notebook utiliza modelos de transferencia de aprendizaje (ResNet50 y EfficientNetB0) para clasificar imágenes de rayos X de tórax como NORMALES o con NEUMONÍA.

In [ ]:
# 📦 Paso 1: Importar librerías
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, EfficientNetB0
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
import cv2
from tensorflow.keras.preprocessing import image
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# 🗂 Paso 2: Definir rutas del dataset
data_dir = "chest_xray"
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")
test_dir = os.path.join(data_dir, "test")

In [ ]:
# 🧹 Paso 3: Generadores de datos
IMG_SIZE = 224
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    preprocessing_function=resnet_preprocess,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
)

val_test_datagen = ImageDataGenerator(preprocessing_function=resnet_preprocess)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="binary",
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=False,
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=False,
)

In [ ]:
# 🧠 Paso 4: Función para construir modelos con Transfer Learning
def build_model(base_model, preprocess_fn, name):
    input_tensor = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = preprocess_fn(input_tensor)
    base = base_model(include_top=False, weights='imagenet', input_tensor=x)
    base.trainable = False
    x = GlobalAveragePooling2D()(base.output)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=input_tensor, outputs=output, name=name)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# 🔁 Paso 5: Entrenamiento
def train_model(model, name):
    checkpoint_path = f"{name}_best_model.h5"
    callbacks = [
        EarlyStopping(patience=3, restore_best_weights=True),
        ModelCheckpoint(checkpoint_path, save_best_only=True)
    ]
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=10,
        callbacks=callbacks
    )
    return model, history

In [ ]:
# 🚀 Entrenar modelos
resnet_model = build_model(ResNet50, resnet_preprocess, "ResNet50")
efficientnet_model = build_model(EfficientNetB0, efficientnet_preprocess, "EfficientNetB0")

resnet_model, resnet_history = train_model(resnet_model, "resnet")
efficientnet_model, efficientnet_history = train_model(efficientnet_model, "efficientnet")

In [ ]:
# 📈 Paso 6: Evaluar modelos
def evaluate_model(model, name):
    print(f"Evaluando {name}")
    preds = model.predict(test_generator)
    y_pred = (preds > 0.5).astype(int)
    y_true = test_generator.classes

    print(classification_report(y_true, y_pred, target_names=["NORMAL", "PNEUMONIA"]))
    cm = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=["NORMAL", "PNEUMONIA"],
                yticklabels=["NORMAL", "PNEUMONIA"])
    plt.title(f'Matriz de Confusión: {name}')
    plt.xlabel('Predicción')
    plt.ylabel('Verdadero')
    plt.show()

In [ ]:
evaluate_model(resnet_model, "ResNet50")
evaluate_model(efficientnet_model, "EfficientNetB0")

In [ ]:
# 🔍 Paso 7: Visualización Grad-CAM
def get_img_array(img_path):
    img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    array = image.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array

def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    grad_model = Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, 0]
    grads = tape.gradient(loss, conv_outputs)[0]
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1))
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def display_gradcam(img_path, model, layer_name="conv5_block3_out"):
    img_array = get_img_array(img_path)
    heatmap = make_gradcam_heatmap(img_array, model, layer_name)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    heatmap = cv2.resize(heatmap, (IMG_SIZE, IMG_SIZE))
    heatmap = np.uint8(255 * heatmap)
    heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(img, 0.6, heatmap_color, 0.4, 0)

    plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
    plt.title("Grad-CAM")
    plt.axis('off')
    plt.show()

In [ ]:
# Mostrar activación Grad-CAM para una imagen del test set
test_img_path = test_generator.filepaths[0]
display_gradcam(test_img_path, resnet_model, "conv5_block3_out")